# CoastVision Example: Waikiki
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 40px;">
        <p>CoastVision is an open-source Python framework geared towards generating satellite-derived shorelines (SDS) in <a href='https://developers.planet.com/docs/data/planetscope/'>PlanetScope</a> imagery. The image the right is two CoastVision SDS superimposed over drone imagery from the same week. This notebook provides an end-to-end example for Waikiki Beach Hawaiʻi.</p>
        <ol>
            <li>Download PlanetScope imagery for a given area of interest (AOI) and timeframe</li>
            <li>Image co-registration</li>
            <li>Shoreline extraction</li>
            <li>Tidal Correction</li>
            <li>QAQC</li>
        </ol>
    </div>
    <div>
        <img src="media\coastvision_shoreline_on_drone_hanaumapng.png" alt="SDS superimposed on high-resolution drone imagery" style="max-width: 100%; height: 100%;">
    </div>
</div>

# Import Libraries
If you have not yet set up environment in conda using `coastvision.yml`
```
cd path/to/CoastVision
conda env create -f coastvision.yml
conda activate coastvision
```


In [2]:
from planetscopeAPI import PlanetScopeOrdersAPI
from coastvision import coastvisionRun, setup, coastvisionCoreg
import pandas as pd
import os

# 1. Download PlanetScope Imagery
<a href='https://developers.planet.com/docs/data/planetscope/'>PlanetScope</a> is a satellite constellation operated by <a href='https://www.planet.com/'>Planet Labs Inc.</a> The PlanetScope constellation is made up of roughtly 130 satellites, capable of imageing the entire land surface of Earth with daily revisit times and 3 meter spatial resolution. The imagery has four bands: red, green, blue, and near-infrared.

In the following cell, information regarding your site (beach or stretch of coastline) should be entered which will be used to download applicable satellite imagery from Planet
1. First enter (lat, long) coordinates (`coords`) creating an AOI (this can be any polygon) around a beach or coastline stretch you are interested in.
2. Next, create the `sitename` and `region` for the site
3. Enter a start and end date. Imagery from between and during these dates will be downloaded for the given API. Date format: `YYYY-MM-DD`
4. Downloading PlanetScope imagery requires an API key. If you do not have a Planet account, you can create one following these steps: <a href='https://www.planet.com/get-started/'>Get Started with Planet</a>

To access your API key log into <a href='https://www.planet.com/'>Planet</a> and navigaet to "My Settings" (see image below).

<img src="media\api_key_planet.JPG" alt="API key in settings" style="max-width:70%">



In [8]:
# create AOI coords (this can be a poly gone with any number of points)
coords = [ 
    [-86.01436311872813,45.04576939601685],
    [-85.96011812361094,45.04795250093743],
    [-85.93127901228281,45.05886677577595],
    [-85.96938783796641,45.07511305896126],
    [-86.02706606062266,45.06274690486688],
    [-86.01436311872813,45.04576939601685]
    ]

region = 'michigan'
sitename = 'northmanitouisland'
setup.create_site_dict_json_for_API(
    site_name=sitename,
    region = region,
    aoi=coords,
    start_date="2018-17-14", # dont know dates yet
    end_date="2018-18-22")

setup.write_api_key_file(api_key='Your PlanetScope API Key', overwrite=False) # this creates a text file that contains your API key and is referenced by PlaneScopeOrdersAPI, with overwrite=False this will not overwrite an existing API key file

In [4]:
## downloading imagery takes some time as it's being requested and processed through Planet. For larger projects, leave for multiple hours or a day. 
## For testing purposes 60 days takes approximately 18 min

import importlib
importlib.reload(PlanetScopeOrdersAPI)
API = PlanetScopeOrdersAPI.PlanetScopeAPIOrder(selectSites=False, printPolling=True) # initalizing the class variable
API.get_all_data()

{'australia': {'narrabeen': {'item_type': 'PSScene', 'geometry_filter': {'type': 'GeometryFilter', 'field_name': 'geometry', 'config': {'type': 'Polygon', 'coordinates': [[[151.2957545, -33.7012561], [151.297557, -33.7388075], [151.312234, -33.7390216], [151.311204, -33.701399], [151.2957545, -33.7012561]]]}}, 'date_range_filter': {'type': 'DateRangeFilter', 'field_name': 'acquired', 'config': {'gte': '2018-11-14T00:00:00.000Z', 'lte': '2018-12-22T00:00:00.000Z'}}, 'cloud_cover_filter': {'type': 'RangeFilter', 'field_name': 'cloud_cover', 'config': {'lte': 0.3}}}}, 'hawaii': {'kepuhibay': {'item_type': 'PSScene', 'geometry_filter': {'type': 'GeometryFilter', 'field_name': 'geometry', 'config': {'type': 'Polygon', 'coordinates': [[[-157.25222755167223, 21.183534287898937], [-157.2508542606566, 21.18271396879255], [-157.2499315807555, 21.182753984464306], [-157.248944527838, 21.183114125022797], [-157.24750686380602, 21.184934822195384], [-157.24677730295397, 21.187395728889875], [-157.2

KeyboardInterrupt: 

# 2. Create Transects and Reference Shoreline
## 2.1 Transects
Coastal change is often measured through a series of shore normal transects. The intersection of SDS are computed and used to measure erosion and accretion along these transects. In this case, the intersections are saved in a dataframe of the following structure:

<body>
    <table border="1">
        <tr>
            <th>Timestamp</th>
            <th>Transect Label 1</th>
            <th>Transect Label 2</th>
            <th>...</th>
            <th>Transect label n</th>
            <!-- Add more headers as needed -->
        </tr>
        <tr>
            <td>2019-03-03 20:43:11</td>
            <td>Distance along transect (m)</td>
            <td>...</td>
            <td>...</td>
            <td>...</td>
            <!-- Add more data cells as needed -->
        </tr>
        <!-- Add more rows as needed -->
    </table>
</body>

The distance along each transect is the distance between the start (landward end) of the transect and where the shoreline intersects it.

*Note:* if you are only interested in shoreline contours and not transect intersections provide `just_extract_shoreline=True` in the `coastvisionRun.CoastVisionRun()` class initialization in 'Initialize CoastVision "Run" Class' below

## 2.2 Reference Shoreline
For quality control, a reference shoreline can be hand digitized. Then given `max_dist_from_sl_ref = max distance in meters` supplied to `coastvision.CoastVisionRun()` all extracted shoreline segments out of this range are discarded. 
*Note:* This is recommended but not necessary.


These should be saved in `user_inputs/<region>/<sitename>` as `<sitename>_transects.geojson` and `<sitename>_shoreline.geojson`. See <a href='https://github.com/Climate-Resilience-Collaborative/CoastVision/blob/aeff3868cea3c3320cc8c6d074d14d4ef581fab5/media/How%20to%20make%20transects%20and%20reference%20shorelines%20for%20CoastVision.pdf'>`media/How to make transects and reference shorelines for CoastVision.pdf`</a> for how to create these files.


<hr>

# 3. Initialize CoastVision "Run" Class
This class represents a site and is used to extract shorelines and compute transect intersections for that site. If you're only interested in the shorelines themselves (not the transect intersections) set `just_extract_shoreline` to `True`. Then `coastvision_run_class.run_shoreline_extraction()` will return a dictionary where each item is a shoreline contour and each key is the PlanetScope image ID and you are done there. Otherwise continue past **Section 5** to complete tidal corrections and QAQC on transect intersections. If data `data_products=True` plots are generated for each image showing the image segmentation, shoreline contour, and transect intersections (**Section 5** has an example of this plot).

In [9]:
coastvision_run_class = coastvisionRun.CoastVisionRun(region=region, sitename=sitename, just_extract_shoreline=False, data_products=False)
print(f'There are {len(coastvision_run_class)} tiff images for site: {sitename}')

There are 12 tiff images for site: waikiki


# 4. Image Co-registration
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 40px;">
        <p>Satellite images need to be accurately registered, meaning that they must align correctly with one another and with real-world coordinates. <a href="https://pypi.org/project/arosics/">AROSICS</a> an open-source Python package is used to co-register images to reduce error caused by image missalignments. In the image below the right pane shows reduced image offsets after AROSICS co-registration. </p>
    </div>
    <div>
        <img src="media/arosics_logo.png" alt="Tidal Effect Example" style="max-width: 100%; height: auto;">
    </div>
</div>

![Co-registration Example](media/co-registration.gif)

**co-registration is optional!** 
<p> It makes a small improvement at reef-lined beaches, but the processing takes a long time. For large projects, leave for hours/overnight. For test example, 37 tiffs takes ~ 20min. <p>

**Select a reference image** and copy the .tif file into 'user_inputs/region/sitename' (without renaming it). Otherwise, the function will select the first tiff file in our files as reference

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 
coastvisionCoreg.coreg_site(region, sitename, grid_res=50, start=0)

# 5. Extract Shorelines and Compute Transect Intersections

The following function runs through the downloaded satellite imagery and does the following:

1. Segment image into land and water
2. Extract shoreline
3. Compute shoreline intersection with transects


<img src="media/stages_plot.jpg" alt="Stages Plot">


This function saves the `intersection_df` at the following path: `outputs/<region/<sitename>_transect_intersections.csv`

In [61]:
intersection_df = coastvision_run_class.run_shoreline_extraction()
if type(intersection_df) == pd.DataFrame:
    display(intersection_df.head(2))
    print(intersection_df.shape)
else:
    print('\nThis means just_extract_shoreline was set to True in the coastvisionRun.CoastVisionRun declaration meaning coastvision_run_class.run_shoreline_extraction() returns a dictionary where each item is a list of coordinates making up a shoreline contour')
    print('Three example shoreline contour points:')
    print(next(iter(intersection_df.values()))[0:3, :])
    print('You\'re done with this example notebook at this stage the rest is for transect intersections')

 waikiki 66.67 percent progress 20240302_211609_83_24e8
skipping this image because it doesn't contain any of the sl ref: 20240302_211609_83_24e8_3B_AnalyticMS_toar_clip.tif

 waikiki 100.0 percent progress 20240302_211617_76_24cb

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
2024-03-01 20:24:30.650,30.169258,41.488963,36.003647,30.653799,26.997789,27.236986,25.909892,25.658780,25.384914,26.177928,34.026438,44.009750,35.821973,35.226834,42.822889,42.398397,36.936619,31.419624,28.876974,21.764293
2024-03-02 21:16:17.760,22.177238,45.686780,NaN,NaN,NaN,NaN,21.047596,23.208589,22.195224,23.083457,29.775743,38.059083,28.917474,28.584743,35.920535,38.407480,33.173116,28.990232,25.243214,19.707945


(5, 20)


# 6. Tidal Corrections

## 6.1 Tidal Effects on Horizontal Shoreline Position
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 20px;">
        <p>CoastVision shoreline accuracy can be improved by correcting for horizontal shifts in shoreline position due to tidal changes. The example to the right from Cadíz, Spain shows how large horizontal changes from tidal shifts can be. Using tide level data, corrections can be made to where the shoreline position would be given mean sea level removing noise added by tidal fluctuations. <p>The refernece elevation is selected by the user and relative to Local Mean Sea Level. In most cases 0 m is a good start. This will be the elevation all shorelines are corrected back to. For example, if the tide height is 0.3 m above LMSL, assuming a linear beach slope, the shoreline will be pushed seaward back to LMSL</p>
    </div>
    <div>
        <img src="media/tide_horizontal_shift_example.gif" alt="Tidal Effect Example" style="max-width: 100%; height: auto;">
    </div>
</div>


<img src="media\tidal_horizontal_effect_figure.png" alt="Tidal Effect Figure">




#### ⚠️ **Complete only one of the tidal corrections below** (if you do both, it will save the most recent tidal correction)
The tidal corrected dataframe is saved as here: `outputs/<region/<sitename>_intersections_tidally_corrected_<reference_elevation>m.csv`


## 6.2 Tidal Corrections via Global Tide Model
The Finite Element Solution (FES)2014 numerical model can be used to calculate ocean tides at any time for any coastal region. These ocean tide height predictions can be used along with a beach slope estimate to compute the horizontal shoreline position correction. The model is quite larges (~8GB) but can be set up on your local computer by downloading it via AVISO. K. Vos and [CoastSat.slope](https://github.com/kvos/CoastSat.slope) provides very helpful documentation for setting up FES2014. See [documentation](https://github.com/kvos/CoastSat.slope/blob/master/doc/FES2014_installation.md) on how to download and use this. 
<p> If you have FES2014 set up and downloaded, use the code below. Identify a coordinate ~1-2 miles offshores using, for example, Google Maps and input it in "offshore_coord". This will be the approximate location of your FES2014 model grid the tides are calculated in (avoid points close to shore as the model can be sporadic in nearshore environments). 

##### ⚠️ **if you don't have the global tide model setup, use local tide gauge data or skip section 6**

In [5]:
fes2014_path = os.path.join(os.getcwd(), 'aviso-fes-main', 'data', 'fes2014')
offshore_coord = [-157.8497, 21.2569]

tidal_corrected_df = coastvision_run_class.tidal_correction_FES2014(fes2014_path, offshore_coord, reference_elevation=0, beach_slope=0.12)
display(tidal_corrected_df.head(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dates,,,,,,,,,,,,,,,,,,,,
2024-01-02 20:18:51.140,30.522349,38.691724,33.013779,28.156301,26.919788,29.184281,29.980206,27.295123,24.417355,25.064117,58.151221,76.052163,67.275611,87.450782,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03 21:01:09.200,24.985083,39.966882,34.480846,29.376896,26.705997,27.485654,27.351242,26.993282,25.279746,24.334678,31.524889,39.963097,30.568290,33.681680,42.955568,42.179545,34.851677,28.732497,28.959894,22.995531


## 6.3 Tidal Corrections via Local Tide Gauge Data
If local tide gauge data is available this can be used rather than FES predictions. point to a path with your tide gauge data. View the example tide gauge data for Honolulu for formatting



In [6]:
# load tide gauge csv
tidegauge = pd.read_csv(os.path.join(os.getcwd(), 'user_inputs', region, sitename, 'honolulutidegauge.csv'), index_col=0, parse_dates = True)
tidegauge.index.name = 'dates'

# do tidal correction 
tidal_corrected_df = coastvision_run_class.tidal_correction_tidegauge(tidegauge, reference_elevation=0, beach_slope=0.12)
display(tidal_corrected_df.head(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dates,,,,,,,,,,,,,,,,,,,,
2024-01-02 20:18:51.140,30.959318,39.128693,33.450748,28.593270,27.356757,29.621249,30.417175,27.732092,24.854324,25.501086,58.588190,76.489132,67.71258,87.887751,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03 21:01:09.200,25.798753,40.780552,35.294516,30.190566,27.519667,28.299324,28.164912,27.806952,26.093416,25.148348,32.338559,40.776767,31.38196,34.495351,43.769239,42.993215,35.665347,29.546167,29.773564,23.809201


# 7. QAQC
There are many sources of error in satellite-derived shorelines, from faulty image classification or image misalignment to uncertainty related to tides and wave swash. While the wealth of data (PlanetScope has near-daily revisit time) enables us to deduce clear shoreline change signals despite much of this noise, it is still important to remove outliers. Below median filtering is used to remove outliers that are not within the `limit` (in meters) of the median (`median - limit < x < median + limit`). This filtered transect intersection dataframe is saved `outputs/<region/<sitename>_QAQC_transect_interesections.csv`.

In [7]:
QAQC_df = coastvision_run_class.intersection_QAQC(limit=30)
display(QAQC_df.head(2))

before QAQC: shape (10, 20) number of nans 20
after QAQC: shape (10, 20) number of nans 22


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dates,,,,,,,,,,,,,,,,,,,,
2024-01-02 20:18:51.140,29.854513,38.023888,32.345942,27.488464,26.251952,28.516444,29.312369,26.627287,23.749518,24.396280,57.483384,NaN,66.607774,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03 21:01:09.200,24.710607,39.692406,34.206370,29.102420,26.431521,27.211178,27.076766,26.718805,25.005270,24.060202,31.250413,39.688621,30.293814,33.407204,42.681092,41.905069,34.577201,28.458021,28.685418,22.721055
